In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import argparse
import warnings
import logging
from os.path import join
from collections import defaultdict
sys.path.append('../')
warnings.filterwarnings("ignore")

from coffea import processor, util
import hist
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_data_vs_mc
from azh_analysis.utils.parameters import get_lumis
import warnings
warnings.filterwarnings('ignore')
#hep.style.use(["CMS", "fira", "firamath"])

mass = "300" 
years = ["2018", "2017", "2016preVFP", "2016postVFP"] #["2016preVFP", "2016postVFP", "2017", "2018"]
lumis = get_lumis()
lumi = sum([lumis[year] for year in years])
if len(years)==4: lumi=lumis["Run 2"]
if len(years)==2: lumi=lumis["2016"]

variables = ["m4l", "mll", "mtt", "pt", "met"]
data, data_ss, mc, ggA, bbA = {}, {}, {}, {}, {}
for var in variables:
    data[var] = sum([sum(util.load(f"../output_test/data_UL_{year}_OS_ub.coffea")[var].values()) for year in years])
    data_ss[var] = sum([sum(util.load(f'../output_test/data_UL_{year}_SS_ub_relaxed.coffea')[var].values()) for year in years])
    mc[var] = sum([sum(util.load(f"../output_test/MC_UL_{year}_all_OS.coffea")[var].values()) for year in years])
    ggA[var] = sum([
        sum([v for k, v in util.load(f"../output_test/signal_UL_{year}_all_OS.coffea")[var].items() if (mass in k) and ("GluGlu" in k)]) 
        for year in years
    ])
    bbA[var] = sum([
        sum([v for k, v in util.load(f"../output_test/signal_UL_{year}_all_OS.coffea")[var].items() if (mass in k) and ("GluGlu" not in k)]) 
        for year in years
    ])


    
cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

plot = True
if len(years)==4:
    outdir = f"../plots/data-mc/all"
elif len(years)==2 and "2016postVFP" in years:
    outdir = f"../plots/data-mc/2016"
elif len(years)==1:
    outdir = f"../plots/data-mc/{years[0]}"
else:
    year_str = years[0]
    for year in years[1:]: year_str = year_str + f"_{year}"
    outdir = f"../plots/data-mc/{year_str}"
    
print(mc["m4l"])

In [ ]:
var = "m4l"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0-btags", 1: "btags"}.items():
        for mass_type in ["cons"]:
            mc_s, data_s, data_ss_s, ggA_s, bbA_s = mc[var], data[var], data_ss[var], ggA[var], bbA[var]
            mc_s = (
                mc_s[:, "ee" + cat, bcat, "nom", mass_type, :] +
                mc_s[:, "mm" + cat, bcat, "nom", mass_type, :]
            )
            data_s = (
                data_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            data_ss_s = (
                data_ss_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_ss_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            ggA_s = (
                ggA_s[:, "ee"+cat, bcat, "nom", mass_type, :] + 
                ggA_s[:, "mm"+cat, bcat, "nom", mass_type, :]
            )
            bbA_s = (
                bbA_s[:, "ee"+cat, bcat, "nom", mass_type, :] + 
                bbA_s[:, "mm"+cat, bcat, "nom", mass_type, :]
            )

            var_labels = {
                "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                "raw": r"$m_{ll\tau\tau}^\mathrm{raw}$ [GeV]",
            }
            plot_data_vs_mc(
                data_s, mc_s, "mass", 
                cat_label=cat_label, var_label=var_labels[mass_type], btag_label=btag_label,
                outfile=join(outdir, f"{cat}_{bcat}_{var}_{mass_type}.pdf") if plot else None,
                logscale=True,
                year=None,
                lumi=lumi,
                ggA=ggA_s, ggA_sigma=5, ggA_mass=mass,
                bbA=bbA_s, bbA_sigma=5, bbA_mass=mass,
                ylim=[0.1, 30],
                data_ss=data_ss_s,
                blind=True,
                blind_range=None,
           )

In [ ]:
# mll per btag

var = "mll"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0-btags", 1: "btags"}.items():
        mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
        mc_s = (
            mc_s[:, "ee" + cat, bcat, "nom", :] + 
            mc_s[:, "mm" + cat, bcat, "nom", :]
        )
        data_s = (
            data_s[:, "ee" + cat, bcat, "none", :] + 
            data_s[:, "mm" + cat, bcat, "none", :]
        )
        data_ss_s = (
            data_ss_s[:, "ee" + cat, bcat, "none", :] + 
            data_ss_s[:, "mm" + cat, bcat, "none", :]
        )
        plot_data_vs_mc(
            data_s, 
            mc_s, 
            "mll", cat_label=cat_label, var_label=r"$m_{ll}$ [GeV]", btag_label=btag_label,
            outfile=join(outdir, f"{cat}_{bcat}_{var}.pdf") if plot else None,
            lumi=lumi,
            year=None,
            data_ss=data_ss_s,
            blind=True,
            blind_range=None,
            #blind=False,
            #blind_range=[4,6],
        )

In [ ]:
# mll all btags

var = "mll"
for cat, cat_label in cat_labels.items():
    mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
    mc_s = (
        mc_s[:, "ee" + cat, ::sum, "nom", :] + 
        mc_s[:, "mm" + cat, ::sum, "nom", :]
    )
    data_s = (
        data_s[:, "ee" + cat, ::sum, "none", :] + 
        data_s[:, "mm" + cat, ::sum, "none", :]
    )
    data_ss_s = (
        data_ss_s[:, "ee" + cat, ::sum, "none", :] + 
        data_ss_s[:, "mm" + cat, ::sum, "none", :]
    )
    plot_data_vs_mc(
        data_s, 
        mc_s, 
        "mll", cat_label=cat_label, var_label=r"$m_{ll}$ [GeV]", btag_label="all btags",
        outfile=join(outdir, f"{cat}_{var}.pdf") if plot else None,
        lumi=lumi,
        year=None,
        data_ss=data_ss_s,
        blind=True,
        blind_range=None,
    )

In [ ]:
# mtt per btag

var = "mtt"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0-btags", 1: "btags"}.items():
        for mass_type in ["raw", "corr", "cons"]:
            blind_ranges = {"raw": [2,7], "corr": [2,9], "cons": [8, 12]}
            mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
            mc_s = (
                mc_s[:, "ee" + cat, bcat, "nom", mass_type, :] + 
                mc_s[:, "mm" + cat, bcat, "nom", mass_type, :]
            )
            data_s = (
                data_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            data_ss_s = (
                data_ss_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_ss_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            plot_data_vs_mc(
                data_s, 
                mc_s, 
                "mtt", cat_label=cat_label, var_label=f"{mass_type} " + r"$m_{tt}$ [GeV]", btag_label=btag_label,
                outfile=join(outdir, f"{cat}_{bcat}_{var}.pdf") if plot else None,
                lumi=lumi,
                year=None,
                data_ss=data_ss_s,
                blind=False,
                blind_range=blind_ranges[mass_type],
            )

In [ ]:
# mtt all btags

var = "mtt"
for cat, cat_label in cat_labels.items():
    mass_type="raw"
    blind_ranges = {"raw": [2,7], "corr": [2,9], "cons": [8, 12]}
    mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
    mc_s = (
        mc_s[:, "ee" + cat, bcat, "nom", "raw", :] + 
        mc_s[:, "mm" + cat, bcat, "nom", "raw", :]
    )
    data_s = (
        data_s[:, "ee" + cat, bcat, "none", "raw", :] + 
        data_s[:, "mm" + cat, bcat, "none", "raw", :]
    )
    data_ss_s = (
        data_ss_s[:, "ee" + cat, bcat, "none", "raw", :] + 
        data_ss_s[:, "mm" + cat, bcat, "none", "raw", :]
    )
    plot_data_vs_mc(
        data_s, 
        mc_s, 
        "mtt", cat_label=cat_label, var_label=r"Visible $m_{tt}$ [GeV]", btag_label="all btags",
        outfile=join(outdir, f"{cat}_{var}.pdf") if plot else None,
        lumi=lumi,
        year=None,
        data_ss=data_ss_s,
        blind=True,
        blind_range=None# blind_ranges[mass_type],
    )

In [ ]:
# met per btag category

var = "met"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0-btags", 1: "btags"}.items():
        mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
        mc_s = (
            mc_s[:, "ee" + cat, bcat, "nom", :] + 
            mc_s[:, "mm" + cat, bcat, "nom", :]
        )
        data_s = (
            data_s[:, "ee" + cat, bcat, "none", :] + 
            data_s[:, "mm" + cat, bcat, "none", :]
        )
        data_ss_s = (
            data_ss_s[:, "ee" + cat, bcat, "none", :] + 
            data_ss_s[:, "mm" + cat, bcat, "none", :]
        )
        plot_data_vs_mc(
            data_s, 
            mc_s, 
            "met", cat_label=cat_label, var_label=r"$E_T^\mathrm{miss}$", btag_label=btag_label,
            outfile=join(outdir, f"{cat}_{bcat}_{var}.pdf") if plot else None,
            lumi=lumi,
            year=None,
            data_ss=data_ss_s,
            blind=False,
            blind_range=None, #[4,6],
        )

In [ ]:
# met all btags

var = "met"
for cat, cat_label in cat_labels.items():
    mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
    mc_s = (
        mc_s[:, "ee" + cat, ::sum, "nom", :] + 
        mc_s[:, "mm" + cat, ::sum, "nom", :]
    )
    data_s = (
        data_s[:, "ee" + cat, ::sum, "none", :] + 
        data_s[:, "mm" + cat, ::sum, "none", :]
    )
    data_ss_s = (
        data_ss_s[:, "ee" + cat, ::sum, "none", :] + 
        data_ss_s[:, "mm" + cat, ::sum, "none", :]
    )
    plot_data_vs_mc(
        data_s, 
        mc_s, 
        "met", cat_label=cat_label, var_label=r"$E_T^\mathrm{miss}$", btag_label="all btags",
        outfile=join(outdir, f"{cat}_{var}.pdf") if plot else None,
        lumi=lumi,
        year=None,
        data_ss=data_ss_s,
        blind=True,
        blind_range=None, #[4,6],
    )

In [ ]:
var = "pt"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0-btags", 1: "btags"}.items():
        for leg in ['1', '2', '3', '4']:
            mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
            mc_s = (
                mc_s[:, "ee" + cat, bcat, "nom", leg, :] + 
                mc_s[:, "mm" + cat, bcat, "nom", leg, :]
            )
            data_s = (
                data_s[:, "ee" + cat, bcat, "none", leg, :] + 
                data_s[:, "mm" + cat, bcat, "none", leg, :]
            )
            data_ss_s = (
                data_ss_s[:, "ee" + cat, bcat, "none", leg, :] + 
                data_ss_s[:, "mm" + cat, bcat, "none", leg, :]
            )
            plot_data_vs_mc(
                data_s, 
                mc_s, 
                "pt", cat_label=cat_label, var_label=f"Leg {leg} " + r"$p_T$", btag_label=btag_label,
                outfile=join(outdir, f"{cat}_{bcat}_{var}.pdf") if plot else None,
                lumi=lumi,
                year=None,
                data_ss=data_ss_s,
                blind=False,
                blind_range=[4,6],
            )

In [ ]:
var = "pt"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {-1: "all btags"}.items():
        for leg in ['1', '2', '3', '4']:
            mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
            mc_s = (
                mc_s[:, "ee" + cat, ::sum, "nom", leg, :] + 
                mc_s[:, "mm" + cat, ::sum, "nom", leg, :]
            )
            data_s = (
                data_s[:, "ee" + cat, ::sum, "none", leg, :] + 
                data_s[:, "mm" + cat, ::sum, "none", leg, :]
            )
            data_ss_s = (
                data_ss_s[:, "ee" + cat, ::sum, "none", leg, :] + 
                data_ss_s[:, "mm" + cat, ::sum, "none", leg, :]
            )
            plot_data_vs_mc(
                data_s, 
                mc_s, 
                "pt", cat_label=cat_label, var_label=f"Leg {leg} " + r"$p_T$", btag_label=btag_label,
                outfile=join(outdir, f"{cat}_{bcat}_{var}.pdf") if plot else None,
                lumi=lumi,
                year=None,
                data_ss=data_ss_s,
                blind=False,
                blind_range=None,
            )

## Same-Sign Cross-Check

In [ ]:
from collections import defaultdict
import os
import hist
import mplhep as hep
import seaborn as sns
import pandas as pd
from coffea import util
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_var_fake_val
import warnings
warnings.filterwarnings('ignore')
hep.style.use(["CMS", "fira", "firamath"])

data = util.load("../data_UL_2018_SS_03-31.coffea")
print(sum(data["mll"].values()))
cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

In [ ]:
var = "m4l"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for mass_type in ["raw", "corr", "cons"]:
                data_s = sum(data[var].values())
                print(data_s)
                data_s = data_s[:, "ee"+cat, 0, mass_type, ::sum, "none", : ] + data_s[:, "mm"+cat, 0, mass_type, ::sum, "none", : ]
                var_labels = {
                    "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                    "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                    "raw": r"$m_{ll\tau\tau}^\mathrm{raw}$ [GeV]",
                }
                data_s, reducible_s = data_s["data", :], data_s["reducible", :]
                plot_var_fake_val(
                    data_s, reducible_s, "mass", cat_label=cat_label, var_label=var_labels[mass_type], 
                    xscale="log",
                )

In [ ]:
from coffea import util
f = util.load("../output/MC_UL_2018_None_OS_04-24.coffea")["mll"]
f = sum([h for k, h in f.items()])

In [ ]:
import os
import uproot
import numpy as np
import awkward as ak

In [ ]:
base = "/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2016/HZJHToWW_preVFP/"
all_evts = {"-1"}
total = 0
sumw = 0
for f in os.listdir(base):
    if "1.root" in f or "1of3_Electrons" not in f: continue
    print(f)
    a = uproot.open(os.path.join(base, f))
    sumw += a["hWeights;1"].values()[0]
    evt = [e["event"] for e in a["Events"]["event"].arrays()]    
    #evt = set([k["event"] for k in evt])
    #all_evts = all_evts | evt
    #total += len(evt)

In [ ]:
len(all_evts) 
# 654,964 unique events in 2018
# 312,786 unique events in 2017
# 107,650 unique events in 2018 1of3_Electrons
# 51,095 unique events in 2017 1of3_Electrons

In [ ]:
total 
# 700,000 events in 2018
# 334,024 events in 2017
# 107,649 events in 2018 1of3_Electrons
# 51,094 events in 2017 1of3_Electrons

In [ ]:
sumw
# 9,427,016.964911222 in 2018 1of3_Electrons
# 547,282.9858785123 in 2017 1of3_Electrons
# 2,817,679.738914609 in 2016postVFP 1of3_Electrons
# 2,830,472.555003166 in 2016preVFP